In this notebook, we show the regressions to study the relationship between the proposed metrics and the future performance.  
The [split data](#split-data) section shows how we split the publications of each scientist into two parts, where the first part before the split point corresponds to her ‘past’ for computing independent variables, and the second consists of the ‘future’ for an assessment of her performance in the rest of her career as the dependent variable.  
The [regression](#regression) section shows the regression function we build and the results we get.  
(Note: the split-point in the notebook is 10.)

In [1]:
import pandas as pd
import warnings
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as st
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from scipy import stats
import datetime
import tqdm as tq
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import pickle


def save_pkl(path, obj):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)


def load_pkl(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


warnings.filterwarnings("ignore")

# split data

In [2]:
def cut_paper(file_dir1, file_dir2, save_dir, behave):
    '''
    Description：split the publications of each scientist into two parts. (Here the split-point is the first 10 papers.)
    Input: 
    - file_dir1: the directory of the '.csv' file which includes the information of authors and their publications
    - file_dir2: the directory of the '.csv' file which includes the exploratory metrics of each publication
    - save_dir: the saving directory of the split data
    - behave: the impact metrics
    Output: the split data in '.csv'
    '''
    for cut_paper in tq.tqdm(range(10, 11)):
        data = pd.read_csv(file_dir1+'original_aps.csv')
        data = data[data.paperCount > cut_paper]  # split point
        data['next_year'] = data.date.astype('str').str[:4]  # year

        data['genres'] = [[int(gg[:2]) for gg in g[1:-1].replace('\'',
                                                                 '').replace(' ', '').split(',')] for g in data['genres']]

        data.next_year = data.next_year.astype('int')
        data.sort_values(['aid', 'next_year'], inplace=True)
        data['pre_count'] = data.groupby('aid').date.rank(
            method='first')  # sort data by paper publication order
        datak1 = data[data.attempt_number <= cut_paper].groupby(
            ['aid'])  # data before split point
        datak2 = data[data.attempt_number > cut_paper].groupby(
            ['aid'])  # data after split point

        N = datak1.paperDoi.count()  # the number of papers before split point
        N = N.reset_index()
        first_year = datak1.next_year.head(1)  # first year
        N['first_year'] = list(first_year)
        first_gere = datak1.genres.head(1)  # first area
        N['first_genre'] = list(first_gere)
        pre_for = datak1[behave].mean()  # performance before split point
        N['past_logCit'] = list(pre_for)

        next_for = datak2[behave].mean()
        next_for = next_for.reset_index()
        next_count = datak2[behave].count()
        next_for['post_paperCount'] = list(next_count)
        whole_career = datak2['CareerYear'].apply(
            lambda x: x.iloc[-1]-x.iloc[0])
        next_for['post_career'] = list(whole_career)
        next_for['whole_count'] = list(datak2['paperCount'].tail(1))

        N = N.merge(next_for, on=['aid'], how='left')

        # explotary metrics
        switch = pd.read_csv(file_dir2+'switch.csv')
        switch.aid = switch.aid.astype(int)
        switch = switch[switch.aid.isin(data.aid)]
        switch.rename(columns={'N5_es_distance': 'es_distance'}, inplace=True)
        switch.rename(columns={'N5_es': 'es'}, inplace=True)
        switch.es = switch.es.astype(int)
        switch['next_year'] = switch.thisDate.astype('str').str[:4]
        switch.next_year = switch.next_year.astype('int')
        switch.sort_values(['aid', 'next_year'], inplace=True)
        pre_switch = switch[switch.attempt_number <= cut_paper].groupby([
                                                                        'aid'])

        switch2 = pre_switch.es.mean()
        es_distance = pre_switch.es_distance.mean()
        switch2 = switch2.reset_index()
        switch2.aid = switch2.aid.astype('int')
        switch2['past_es_dis'] = list(es_distance)

        N = N.merge(switch2, on=['aid'])  # merge data

        N.rename(columns={'paperDoi': 'past_paperCount', str(
            behave): 'post_logCit', 'es': 'past_es'}, inplace=True)
        N.to_csv(
            save_dir+"split_paper/N_{}_author_info.csv".format(cut_paper), index=0)

In [3]:
cut_paper('../data/regression/', '../data/regression/',
          '../data/regression/', 'logCit')

100%|██████████| 1/1 [00:06<00:00,  6.48s/it]


In [4]:
def cut_career(file_dir1, file_dir2, save_dir, behave):
    '''
    Description：split the publications of each scientist into two parts. (Here the split-point is the first 10 career years.)
    Input: 
    - file_dir1: the directory of the '.csv' file which includes the information of authors and their publications
    - file_dir2: the directory of the '.csv' file which includes the exploratory metrics of each publication
    - save_dir: the saving directory of the split data
    - behave: the impact metrics
    Output: the split data in '.csv'
    '''
    for cut_paper in tq.tqdm(range(10, 11)):
        data = pd.read_csv(file_dir1+'original_aps.csv')
        data = data[data.cyCount > cut_paper]
        data['next_year'] = data.date.astype('str').str[:4]
        data.next_year = data.next_year.astype('int')
        data.sort_values(['aid', 'next_year'], inplace=True)

        data['genres'] = [[int(gg[:2]) for gg in g[1:-1].replace('\'',
                                                                 '').replace(' ', '').split(',')] for g in data['genres']]

        datak1 = data[data.CareerYear <= cut_paper].groupby(['aid'])
        datak2 = data[data.CareerYear > cut_paper].groupby(['aid'])

        N = datak1.paperDoi.count()
        N = N.reset_index()
        first_year = datak1.next_year.head(1)
        N['first_year'] = list(first_year)
        first_gere = datak1.genres.head(1)
        N['first_genre'] = list(first_gere)
        pre_for = datak1[behave].mean()
        N['past_logCit'] = list(pre_for)

        next_for = datak2[behave].mean()
        next_for = next_for.reset_index()
        next_count = datak2[behave].count()
        next_for['post_paperCount'] = list(next_count)
        whole_career = datak2['CareerYear'].apply(
            lambda x: x.iloc[-1]-x.iloc[0])
        next_for['post_career'] = list(whole_career)
        next_for['whole_count'] = list(datak2['paperCount'].tail(1))

        N = N.merge(next_for, on=['aid'], how='left')

        switch = pd.read_csv(file_dir2+'switch.csv')
        switch.aid = switch.aid.astype(int)
        switch = switch[switch.aid.isin(data.aid)]
        switch.rename(columns={'N5_es_distance': 'es_distance'}, inplace=True)
        switch.rename(columns={'N5_es': 'es'}, inplace=True)
        switch.es = switch.es.astype(int)
        switch['next_year'] = switch.thisDate.astype('str').str[:4]
        switch.next_year = switch.next_year.astype('int')
        switch.sort_values(['aid', 'next_year'], inplace=True)
        pre_switch = switch[switch.CareerYear <= cut_paper].groupby(['aid'])
        switch2 = pre_switch.es.mean()
        es_distance = pre_switch.es_distance.mean()
        switch2 = switch2.reset_index()
        switch2.aid = switch2.aid.astype('int')
        switch2['past_es_dis'] = list(es_distance)

        N = N.merge(switch2, on=['aid'])

        N.rename(columns={'paperDoi': 'past_paperCount', str(
            behave): 'post_logCit', 'es': 'past_es'}, inplace=True)
        N.to_csv(
            save_dir+"split_career/cyear_{}_author_info.csv".format(cut_paper), index=0)

In [5]:
cut_career('../data/regression/', '../data/regression/',
           '../data/regression/', 'logCit')

100%|██████████| 1/1 [00:05<00:00,  5.16s/it]


# regression

In [6]:
# get dummay variable for paper areas
def get_dummy(datak):
    list100 = []
    aidlist = []
    for i, g in datak.groupby(['aid']):
        aidlist.append(i)

        this_genres = [np.nan]*100
        i = list(g.first_genre)[0]
        i = i[1:-1]
        i = i.replace('\'', '')
        i = i.replace(' ', '')
        i = i.split(',')
        for t_i in i:
            this_genres[int(t_i[:2])] = 1
        list100.append(this_genres)

    aid_genre = pd.DataFrame(list100)
    aid_genre['aid'] = aidlist
    datak = datak.merge(aid_genre, on=['aid']).drop(columns='first_genre')
    return datak

# calculate normalized coefficients
def cal_norm_k(k, std_x, std_y):
    return k * std_x / std_y


def reg_and_pre(poy, file_dir, select, attribute_y, attributes, dummy_attris, summary, range_list):
    '''
    Description：regression function
    Input:
    - poy: split type, 'p' for 'paper', 'cy' for 'career year'
    - file_dir: the directory of the split data
    - select: True if select scientists who have at least 5 publications before the split points or at least 3 publications after the split points, for less variance.
    - attribute_y: the impact metrics
    - attributes: the exploratory metrics
    - dummy_attris: variables which should be encoded into dummy variables
    - summary: if True, print the regression summary
    - range_list: the range of split point
    Output: 
    - parameters: parameter dict of regression
    - est: regression model
    '''
    N = []
    r2 = []
    mse = []
    rmse = []
    mae = []

    mu_bar = []
    mu = []
    std_bar = []
    std = []

    pvalue = {}
    norm_k_order = []
    coeff = {}
    norm_coeff = {}
    err = {}
    norm_err = {}

    if poy == 'p':
        f_str = "N_{}_author_info.csv"

    elif poy == 'cy':
        f_str = "cyear_{}_author_info.csv"

    for cut_year in range_list:

        predict = pd.read_csv(file_dir+f_str.format(cut_year))

        if select:
            if poy == 'y':
                predict = predict[(predict.past_paperCount >= 5) & (
                    predict.post_paperCount >= 3)]
                behave_list = ['past_logCit', 'past_paperCount']
            if poy == 'p':
                predict = predict[(predict.post_paperCount >= 3)]
                behave_list = ['past_logCit']
            if poy == 'cy':
                predict = predict[(predict.past_paperCount >= 5) & (
                    predict.post_paperCount >= 3)]
                behave_list = ['past_logCit', 'past_paperCount']

        if attributes:
            x_label_1 = attributes + behave_list

        else:
            x_label_1 = behave_list

        predict = predict[x_label_1 +
                          ['aid', 'first_genre', 'first_year']+[attribute_y]]
        predict = predict.dropna()
        if len(predict) == 0:
            continue

        predict = get_dummy(predict)
        predict = pd.concat((predict, pd.get_dummies(
            predict['first_year'], drop_first=True)), axis=1).drop(columns='first_year')
        if attributes:
            attributes_used = attributes.copy()
        if dummy_attris:
            for attri in dummy_attris:
                dummy_df = pd.get_dummies(predict[attri]).drop(
                    columns=max(predict[attri]))
                predict = pd.concat((predict, dummy_df),
                                    axis=1).drop(columns=attri)
                attributes_used += list(dummy_df.columns)
                attributes_used.remove(attri)

        predict = predict.dropna(axis=1, how='all')
        predict.fillna(0, inplace=True)

        x_label = list(set(predict.columns) - set(['aid', attribute_y]))
        X_train = predict[x_label]
        X_train = sm.add_constant(X_train)
        Y_train = predict[attribute_y]
        est = sm.OLS(Y_train, X_train).fit()

        if summary:
            print(est.summary())

        if attributes:
            for a in attributes_used:
                if a not in coeff.keys():
                    pvalue[a] = [float(est.pvalues[a])]
                    coeff[a] = [float(est.params[a])]
                    norm_coeff[a] = [cal_norm_k(
                        float(est.params[a]), X_train[a].std(), Y_train.std())]
                    err[a] = [float(est.bse[a])]
                    norm_err[a] = [cal_norm_k(
                        float(est.bse[a]), X_train[a].std(), Y_train.std())]
                else:
                    pvalue[a].append(float(est.pvalues[a]))
                    coeff[a].append(float(est.params[a]))
                    norm_coeff[a].append(cal_norm_k(
                        float(est.params[a]), X_train[a].std(), Y_train.std()))
                    err[a].append(float(est.bse[a]))
                    norm_err[a].append(cal_norm_k(
                        float(est.bse[a]), X_train[a].std(), Y_train.std()))
        N.append(est.nobs)
        r2.append(est.rsquared_adj)

    parameters = {'r2': r2, 'N': N, 'mse': mse, 'rmse': rmse, 'mae': mae, 'mu_bar': mu_bar, 'mu': mu, 'std_bar': std_bar, 'std': std,
                  'pvalue': pvalue, 'norm_k_order': norm_k_order, 'coeff': coeff, 'norm_coeff': norm_coeff, 'err': err, 'norm_err': norm_err}
    print(parameters.keys())

    return parameters, est


def test(para):
    r2, N, mse, rmse, mae, mu_bar, mu, std_bar, std, pvalue, norm_k_order, coeff, norm_coeff, err, norm_err = para[0].values(
    )
    for key in coeff.keys():
        print('%s The regression coefficient are positive' % key, '%.2f' %
              (sum([c > 0 for c in coeff[key]])/len(coeff[key])))
        print('%s The regression coefficient are significant, p value<=0.05' %
              key, '%.2f' % (sum([p < 0.05 for p in pvalue[key]])/len(pvalue[key])))
    print('Average predicted R2 %.2f ranges %.2f-%.2f' %
          (np.mean(r2), np.min(r2), np.max(r2)))

In [7]:
file_dir = '../data/regression/split_paper/'

para_1 = reg_and_pre('p', file_dir, True, 'post_logCit', [
                     'past_es', 'past_es_dis'], False, True, range(10, 11))
# test(para_1)

                            OLS Regression Results                            
Dep. Variable:            post_logCit   R-squared:                       0.342
Model:                            OLS   Adj. R-squared:                  0.338
Method:                 Least Squares   F-statistic:                     89.40
Date:                Tue, 07 Feb 2023   Prob (F-statistic):               0.00
Time:                        19:45:31   Log-Likelihood:                -12561.
No. Observations:               18154   AIC:                         2.533e+04
Df Residuals:                   18048   BIC:                         2.616e+04
Df Model:                         105                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.7320      0.080      9.123      

In [8]:
para_2 = reg_and_pre('p', file_dir, True, 'post_logCit', [
                     'past_es'], False, True, range(10, 11))
# test(para_2?)

                            OLS Regression Results                            
Dep. Variable:            post_logCit   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     89.57
Date:                Tue, 07 Feb 2023   Prob (F-statistic):               0.00
Time:                        19:45:33   Log-Likelihood:                -12585.
No. Observations:               18154   AIC:                         2.538e+04
Df Residuals:                   18049   BIC:                         2.620e+04
Df Model:                         104                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.6286      0.079      7.964      

In [9]:
file_dir = '../data/regression/split_career/'

para_3 = reg_and_pre('cy', file_dir, True, 'post_logCit', [
                     'past_es', 'past_es_dis'], False, True, range(10, 11))
test(para_3)

                            OLS Regression Results                            
Dep. Variable:            post_logCit   R-squared:                       0.285
Model:                            OLS   Adj. R-squared:                  0.280
Method:                 Least Squares   F-statistic:                     58.71
Date:                Tue, 07 Feb 2023   Prob (F-statistic):               0.00
Time:                        19:45:34   Log-Likelihood:                -10468.
No. Observations:               14526   AIC:                         2.113e+04
Df Residuals:                   14427   BIC:                         2.189e+04
Df Model:                          98                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.7053      0.108     

In [10]:
para_4 = reg_and_pre('cy', file_dir, True, 'post_logCit', [
                     'past_es'], False, True, range(10, 11))
test(para_4)

                            OLS Regression Results                            
Dep. Variable:            post_logCit   R-squared:                       0.283
Model:                            OLS   Adj. R-squared:                  0.279
Method:                 Least Squares   F-statistic:                     58.81
Date:                Tue, 07 Feb 2023   Prob (F-statistic):               0.00
Time:                        19:45:36   Log-Likelihood:                -10486.
No. Observations:               14526   AIC:                         2.117e+04
Df Residuals:                   14428   BIC:                         2.191e+04
Df Model:                          97                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.5913      0.107     